### 교차검증과 그리드서치

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [ ]:

wine.info()  ## info() 함수는 pandas DataFrame 만 적용 가능하다

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [ ]:
wine.describe()

,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

### 검증 세트



```
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(input_array, target_array, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
```

  - train_test_split 함수를 두번 사용하여 기존의 훈련 데이터를 sub 데이터와 val 데이터로 분류한다.
  - 이후 sub 데이터를 학습시키고 val 데이터를 활용하여 모델의 하이퍼 파라미터를 튜닝 한다음 best parameter 를 설정한다.
  - 그 다음 전체 훈련데이터(sub + val) 을 가지고 모델을 다시 학습시키고 테스트 데이터를 적용해서 모델의 성능을 검증한다.


In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

### 교차검증
  - 검증데이터(20%)를 전체 훈련데이터(모델학습에 필요한 데이터)에서 차감하여 훈련데이터를 설정하기에는 데이터 숫자가 너무 적어 데이터 학습에 지장이 있을 것이라는 예상이 될 경우 훈련에 더 많은 데이터를 사용하기 위해서 사용하는 방법

  - 전체 훈련 데이터를 3 등분하여 순서대로 돌아가면서 검증세트로 활용하는 방법(총 3번의 모델학습과 평가가 이루어짐)

  - 그리고 3개의 검증 점수를 평균한 점수를 일반적 검증 점수로 활용하여 모델의 하이터 파라미터를 조정한다.

  - 전체 데이터가 적다면 훈련데이터를 3개 이상으로 나누어서 위와 동일한 방법으로 활용할 수있다.(예시 : 훈련 데이터를 10개로 분할하여 10번의 모델 훈련과 모델평가 후 검증 점수를 활용하여 하이퍼 파라미터를 조정한다.)

  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8454877814123533
0.8415384615384616


In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00484371, 0.00349998, 0.00362325, 0.00367737, 0.00334573]), 'score_time': array([0.00112772, 0.00097632, 0.00106788, 0.00110769, 0.00105143]), 'test_score': array([0.84230769, 0.83365385, 0.84504331, 0.8373436 , 0.8479307 ])}


  - sklearn 의 cross_validate 함수를 활용하여 모델, 훈련 데이터를 넣어주면 기본값(default =5)을 활용하여 5개로 분할 하여 5번 모델을 훈련 평가한다.
  - test_score 는 검증점수로서 각 모델(5개)의 모델의 검증 점수를 배열로 보여준다
  - fit_time 과 score_time은 각 학습과 평가에 걸리는 시간을 나타낸다

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

0.8412558303102096


  - 일반적으로 DL(Deep Learning) 에서는 검증세트, 훈련세트를 활용하며 반드시 교차검증이나 그리드 서치를 요하지 않는다.
    (  
    - 왜냐면 데이터가 너무 충분하기 때문에 굳이 데이터가 적을 경우의 훈련 검증 방법을 사용하지 않는다,
  
    - 또한 딥러닝 자체가 한번 훈련할때 자원을 너무 많이 사용하기 때문에 굳이 훈련과 검증을 여러번 해야되는 교차검증 방법 혹은 그리드 서치 방법등을 사용하기에는 비효율 적이다.
    )

  - 일반적으로 ML(Machine Learning)에서는 검증세트와 훈련세트를 좀더 세부적으로 나눈 교차검증을 활용하거나, 그리드 서치를 하도록 권장된다.

### 분할기를 사용한 교차 검증
  - cross_validata 메소드와 같은 것들을 spliter라고 부르며 만약 교차검증시 데이터를 조금더 많이 분할 하여 검증하고자 한다면 cv를 조정한다.(예시 : 10개의 폴더를 설정하고 싶다면 'cv = 10' 으로 조정한다.)

  - 또 다른 방법으로 cv 파라미터에 스플리터 객체를 설정하여 많은 동작을 상세하게 조정하는 방법이 있다.

  - cross_validate 함수는 기본적으로 KFold() 라는 스플리터 객체를 사용하고 있으며 KFold()는 회귀모델에 사용된다.

  - 분류모델에는 StratifiedKFold() 객체를 사용한다.(stratify 는 train_test_split 함수에서 훈련데이터셋과 테스트데이터셋의 특정 데이터 편중현상이 발생하지 않도록 고르게 분할하는 매개변수이다.)

  - cross_validate 함수를 사용할때 dt(모델)을 입력하면 모델이 회귀모델 혹은 분류모델 일 경우에 따라서 cv 가 자동으로 Kfold(), StratifiedKFold() 로 설정된다.





In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8412558303102096


  - 만약 StratifiedKFold() 객체의 분할 숫자를 10개로 늘리고 싶다면 아래와 같이 StratifiedKFold() 메소드의 n_splits 파라미터를 10으로 설정하여 입력해준다.

  - 참고로 위에서 설정한 K개로 나누고 K번 검증하는 방법을 KFold 라고 하며, K개로 나눌때 특정 데이터의 편중을 없애기 위한 방법이 StratifiedKFold 라고 한다.

In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8335549132947977


### 그리드 서치

  - min_impurity_decrease

    - min_impurity_decrease 노드를 분할하기 위한 불순도 감소의 최소 값을 의미하며, 분할로 인해 불순도 감소가 이 값 이상일 때만 분할이 수행됩니다.

    - 이를 통해 너무 미세한 분할을 방지하고, 모델의 복잡도를 조절할 수 있습니다.

    - min_impurity_decrease는 작은 값을 설정하면 더 많은 분할이 이루어지며, 큰 값을 설정하면 분할이 제한됩니다.


  - max_depth

    - 트리의 최대 깊이를 설정합니다.

    - 트리의 성장을 제한하여 과적합(overfitting)을 방지합니다. 깊이가 깊어질수록 모델이 복잡해지고, 깊이가 얕을수록 모델이 단순해집니다.

    - 작은 값을 설정하면 모델이 덜 복잡해지고, 큰 값을 설정하면 모델이 더 복잡해집니다.

  - min_samples_split

    - 노드를 분할하기 위한 최소 샘플 수를 설정합니다.

    - 노드를 분할하기 위해 필요한 최소 샘플 수를 설정하여, 분할이 지나치게 이루어지는 것을 방지합니다. 작은 값을 설정하면 더 많은 분할이 이루어지고, 큰 값을 설정하면 분할이 제한됩니다.

    - 값이 작으면 더 세밀한 분할이 이루어지며, 값이 크면 덜 세밀한 분할이 이루어집니다.

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


  - GridSearchCV 함수의 n_jobs 의 파라미터는 사용할 CPU의 core 갯수를 의미하며 만약 core가 2개를 사용한다면 n_jobs = 2 를 표시하면된다.

  - n_jobs가 -1 이면 가능한 모든 코어를 전부 사용하여 계산한다는 의미이다.

  - sklearn 라이브러리의 특징은 GridSearchCV() 메소드를 포함한 여러 메소드를 하나의 변수 (여기서는 gs)로 설정하여 다른 회귀모델 혹은 분류모델과 동일하게 fit 메소드로 사용 가능하다는 점이다. (다른 API를 설정하지 않음)



### GridSearchCV의 주요 기능

1. 하이퍼파라미터 탐색: 사용자가 지정한 하이퍼파라미터 그리드를 탐색합니다.

2. 교차 검증 수행: 각 하이퍼파라미터 조합에 대해 교차 검증을 수행하여 모델의 성능을 평가합니다.

3. 최적의 하이퍼파라미터 선택: 교차 검증 결과를 바탕으로 최적의 하이퍼파라미터 조합을 선택합니다.

###주요 파라미터

1. estimator: 하이퍼파라미터 튜닝을 수행할 머신러닝 모델.

2. param_grid: 탐색할 하이퍼파라미터와 그 값들의 조합.

3. cv: 교차 검증 분할 수. 기본값은 5입니다.

4. scoring: 모델 평가를 위한 성능 지표. 예를 들어, accuracy, neg_mean_squared_error 등이 있습니다.

5. n_jobs: 병렬 처리를 위한 CPU 코어 수. -1로 설정하면 모든 가용 코어를 사용합니다.

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])
print(max(gs.cv_results_['mean_test_score']))

[0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]
0.8683865773302731


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.892053107562055


In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(gs.cv_results_['mean_test_score'])
print(max(gs.cv_results_['mean_test_score']))

[0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]
0.8683865773302731


###GridSearchCV를 사용할 수 있는 모델

Scikit-learn의 대부분의 모델은 GridSearchCV를 사용할 수 있습니다. 여기에는 다음과 같은 모델들이 포함됩니다:

1. 회귀 모델 (Regression Models)

  - LinearRegression
  - Ridge
  - Lasso
  - ElasticNet
  - SVR (Support Vector Regression)
  - RandomForestRegressor
  - GradientBoostingRegressor
  - KNeighborsRegressor
  - DecisionTreeRegressor
  - AdaBoostRegressor

2. 분류 모델 (Classification Models)

  - LogisticRegression
  - SVC (Support Vector Classification)
  - RandomForestClassifier
  - GradientBoostingClassifier
  - KNeighborsClassifier
  - DecisionTreeClassifier
  - AdaBoostClassifier
  - NaiveBayes 계열 (GaussianNB, MultinomialNB, 등)
  - MLPClassifier (Multi-layer Perceptron)

3. 클러스터링 (Clustering)

  - KMeans
  - DBSCAN (다만, 매개변수 튜닝은 가능하지만, 교차 검증이 의미가 없을 수 있습니다.)

4. 앙상블 방법 (Ensemble Methods)

  - BaggingClassifier
  - BaggingRegressor
  - VotingClassifier
  - VotingRegressor
  - StackingClassifier
  - StackingRegressor



###GridSearchCV를 사용할 수 없는 모델


1. 미리 정의된 모델 또는 외부 모델

  - Scikit-learn 외부의 모델이나, 미리 정의된 다른 라이브러리의 모델은 직접적으로 호환되지 않을 수 있습니다. 하지만, Scikit-learn의 BaseEstimator와 ClassifierMixin 또는 RegressorMixin을 상속받아 커스터마이즈된 래퍼 클래스를 사용하여 호환할 수 있습니다.

2. 훈련이나 예측 시 비표준 인터페이스를 사용하는 모델

  - Scikit-learn의 표준 인터페이스 (fit, predict 등)를 따르지 않는 모델은 GridSearchCV와 호환되지 않습니다.

### 확률분포선택

In [ ]:
from scipy.stats import uniform, randint  ## 균등분포에서 샘플링하는 클래스,, uniform은 실수, randint는 정수 값을 샘플링한다
rgen = randint(0, 10)
rgen.rvs(10) ## rvs 메소드는 랜덤으로 () 안에 있는 숫자만큼 샘플링 하는 메소드이다.

array([6, 1, 8, 5, 3, 0, 0, 7, 9, 0])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([121,  83, 102,  88,  93,  93, 102, 117, 106,  95]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.47776138, 0.0255728 , 0.4098474 , 0.00249596, 0.70933021,
       0.23417164, 0.3717843 , 0.39016294, 0.47588847, 0.13037519])

## 랜덤 서치 (RandomizedSearch)

  - 일반적으로 데이터의 양이 방대하고 그리드 서치를 사용하기에는 비교해야 되는 모델의 다양한 파라미터에 따른 방대한 모델 숫자를 모두 적용하기 어려울 경우 랜덤(균등)하게 서치하여 모델을 샘플링하여 검증하는 방법

  - sklearn 라이브러리의 RandomizedSearchCV 메소드는 교차검증(cross-validate)의 방법을 이미 포함하고 있다. 결국 GridSearchCV 방법에서 전수조사 방법을 샘플링으로 바꾼 방법으로 이해하면된다,.

  - 깊게 들어가면 랜덤서치에서 조정 가능한 매개변수를 조정하는 툴, 혹은 방법들이 존재하며, 이는 실무적으로 중요하다.

  - RandomizedSearchCV 함수에서 결정트리와 관련된 파라미터에 대해 설명하기로 한다.

  - min_impurity_decrease

    - 이 파라미터는 노드를 분할하기 위한 불순도 감소의 최소 값을 설정합니다.
    
    - 노드를 분할할 때, 분할로 인해 불순도 감소가 이 값 이상일 때만 분할이 수행됩니다.
    
    - 이는 모델이 과도하게 복잡해지는 것을 방지하고, 중요한 분할만 수행되도록 합니다.

  - max_depth

    - 트리의 최대 깊이를 설정합니다

    - 트리의 성장을 제한하여 과적합(overfitting)을 방지합니다.

    - 깊이가 깊어질수록 모델이 복잡해지고, 깊이가 얕을수록 모델이 단순해집니다.

  - min_samples_split

    - 노드를 분할하기 위한 최소 샘플 수를 설정합니다.

    - 노드를 분할하기 위해 필요한 최소 샘플 수를 설정하여, 분할이 지나치게 이루어지는 것을 방지합니다.

    - 작은 값을 설정하면 더 많은 분할이 이루어지고, 큰 값을 설정하면 분할이 제한됩니다.

  - min_samples_leaf

    - 리프 노드(Leaf Node)에 필요한 최소 샘플 수를 설정합니다.

    - 리프 노드에 있어야 하는 최소 샘플 수를 지정하여, 너무 작은 샘플 수로 인해 불안정한 예측을 방지합니다.

    - 작은 값을 설정하면 리프 노드가 더 많이 생성되고, 큰 값을 설정하면 리프 노드가 제한됩니다.

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

0.8928227823744468
{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
# print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))


0.86
